# 数据预处理

在Python中常用的数据分析工具中，我们通常使用pandas软件包。像庞大的Python生态系统中的许多其他扩展包一样，pandas可以与张量兼容。本节我们将简要介绍使用pandas预处理原始数据，并将原始数据转换为张量格式的步骤。我们将在后面的章节中介绍更多的数据预处理技术。

## 读取数据集

首先创建一个文件 `house_tiny.csv`，然后我们用程序进行写入

In [1]:
#!mkdir data && touch data/house_tiny.csv
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')
    f.write('NA,Pave,127500\n')
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

In [2]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


## 处理缺失值

NaN 值就是缺失的值。在实际情况下有两种处理方法：

- 删除他们
- 插值法

下面就介绍插值法。

通过位置索引 iloc，我们将 data 分成 inputs 和 outputs， 其中前者为 data 的前两列，而后者为 data 的最后一列。对于 inputs 中缺少的数值，我们用同一列的**均值**替换“NaN”项

In [3]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


对于“Alley”这一列，它的值有“Pave”和“NaN”两类，这个时候就可以根据这两类分类创建另外两列：

- 第一列是“是否为 pave”：是则赋为 1，反之为 0
- 第二列是“是否为 NaN”：是则赋为 1，反之为 0

实现的方法就是 `pandas.get_dummies()` 方法：

In [4]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


实际上当不止两种类型的情况下也可以使用，但就会生成更多的列了：

Ref: https://blog.csdn.net/maymay_/article/details/80198468

## 转化为张量

In [5]:
import torch

X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))

## 作业

### 删除缺失值最多的列

In [6]:
data_2 = pd.read_csv(data_file)

is_nan = data.isna().sum()
is_nan

NumRooms    2
Alley       3
Price       0
dtype: int64

In [8]:
is_nan_dit = is_nan.to_dict()
max_index = max(is_nan_dit, key=is_nan_dit.get)

print(max_index)
data_2.drop([max_index], axis=1)
data

Alley


,NumRooms,Price
0,NaN,127500
1,2.0,106000
2,4.0,178100
3,NaN,140000


Ref:

- 删除 Pandas 中的行或列：https://blog.csdn.net/songyunli1111/article/details/79306639

- Pandas 清洗数据（统计 NaN）：https://blog.51cto.com/u_15162069/2919997